# Progetto di Foundation of computer science

### Corso di laurea in Data Science A.A 2019/2020

### Simone D'Amico - 850369

In [1]:
# Import delle libreire usate
import pandas as pd
import numpy as np

In [2]:
# Caricamento del tempo di esecuzione di ogni cella
%load_ext autotime

# Definizione del formato per i numeri decimali, si mostra fino alla terza cifra decimale inclusa
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

### Caricamento dei dataset utilizzati

In [3]:
# Per motivi di tempo e di memoria sono caricate dai dataset solo le colonne usate per i vari task

df_loans_lenders = pd.read_csv('loans_lenders.csv')

use_col_loans =['loan_id', 'loan_name', 'disburse_time', 'planned_expiration_time', 
                'country_name', 'loan_amount', 'country_code']
date_type = ['disburse_time', 'planned_expiration_time']
df_loans = pd.read_csv('loans.csv', usecols=use_col_loans, parse_dates=date_type)


use_col_lenders =['permanent_name', 'city', 'state', 'country_code']
df_lenders = pd.read_csv('lenders.csv', usecols=use_col_lenders)


used_col_country = ['country_name', 'country_code', 'population', 'population_below_poverty_line']
df_country_stats = pd.read_csv('country_stats.csv', usecols=used_col_country)

time: 23 s


### 1. Normalize the loan_lenders table. In the normalized table, each row must have one `loan_id` and one lender.

In [4]:
# Partendo dal dataset df_loans_lenders si usano come indici loan_id, così si conservano durante l'esecuzione. 
# Ogni valore di lenders viene trasformato in una lista e, grazie a explode, 
# si ottengono le coppie (loan, specifico lender)
df_loan_lender_norm = df_loans_lenders.set_index(['loan_id']).apply(
    lambda x: x.str.split(', ').explode()).reset_index()

time: 40.7 s


In [5]:
df_loan_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


time: 34.1 ms


### 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [6]:
# Si calcola la differenza in giorni tra le colonne delle date e si considerano i giorni
df_loans['diff_days'] = (df_loans['planned_expiration_time'] - df_loans['disburse_time']).dt.days

time: 97.9 ms


In [7]:
df_loans.head()

,loan_id,loan_name,loan_amount,country_code,country_name,planned_expiration_time,disburse_time,diff_days
0,657307,Aivy,125.000,PH,Philippines,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53.000
1,657259,Idalia Marizza,400.000,HN,Honduras,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96.000
2,658010,Aasia,400.000,PK,Pakistan,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37.000
3,659347,Gulmira,625.000,KG,Kyrgyzstan,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34.000
4,656933,Ricky\t,425.000,PH,Philippines,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57.000


time: 28.8 ms


In [8]:
# Per verificare che le righe della colonna diff_days siano mancanti se almeno una delle colonne 
# planned_expiration_time o disburse_time è mancante si può usare la funzione info()

df_loans[df_loans['planned_expiration_time'].isnull()].info()

print('\n=================================================================\n')

df_loans[df_loans['disburse_time'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371834 entries, 64 to 1419523
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   loan_id                  371834 non-null  int64              
 1   loan_name                355759 non-null  object             
 2   loan_amount              371834 non-null  float64            
 3   country_code             371834 non-null  object             
 4   country_name             371834 non-null  object             
 5   planned_expiration_time  0 non-null       datetime64[ns, UTC]
 6   disburse_time            371832 non-null  datetime64[ns, UTC]
 7   diff_days                0 non-null       float64            
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(3)
memory usage: 25.5+ MB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2813 entries, 31106 to 1419564
Data columns (total 8 columns):
 #   Column     

### 3. Find the lenders that have funded at least twice.

In [9]:
# Si raggruppa per lender e si ottiene il numero di record di ogni gruppo
count = df_loan_lender_norm.groupby(['lenders']).size()

# Si selezionano solo i lenders che hanno finanziato due o più volte 
count = count[count >=2]

# Si definisce un dataset per mostrare il risultato
df_count = count.reset_index(name='count')

time: 12.5 s


In [60]:
df_count.head()

,lenders,count
0,000,40
1,00000,39
2,0002,70
3,0101craign0101,71
4,0132575,4


time: 11.8 ms


### 4. For each country, compute how many loans have involved that country as borrowers.

In [10]:
# Si può usare value_count applicato alla colonna country_name per contare le occorrenze di ogni paese 
# ottienendo un dataframe 
df_count_loans = df_loans['country_name'].value_counts().reset_index()

df_count_loans = df_count_loans.rename(columns= {'index': 'country_name', 'country_name': 'count'})

time: 181 ms


In [11]:
df_count_loans.head()

,country_name,count
0,Philippines,285336
1,Kenya,143699
2,Peru,86000
3,Cambodia,79701
4,El Salvador,64037


time: 8.69 ms


### 5. For each country, compute the overall amount of money borrowed.

In [12]:
# Si raggruppano i loans per country_name e si calcola la somma di loan_amount per ogni gruppo
df_overall_borrowed = df_loans.groupby('country_name')['loan_amount'].sum().reset_index(name='tot_borrow')

time: 122 ms


In [13]:
df_overall_borrowed.head()

,country_name,tot_borrow
0,Afghanistan,1967950.000
1,Albania,4307350.000
2,Armenia,22950475.000
3,Azerbaijan,14784625.000
4,Belize,150175.000


time: 9.34 ms


### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [14]:
# Si calcola la somma totale dei prestiti
tot_borrow = df_overall_borrowed['tot_borrow'].sum()

# Si calcola la percentuale di prestiti di ogni stato rispetto alla somma totale
df_overall_borrowed['perc_borrow'] = (df_overall_borrowed['tot_borrow']/tot_borrow)*100

time: 13.1 ms


In [15]:
df_overall_borrowed.head()

,country_name,tot_borrow,perc_borrow
0,Afghanistan,1967950.000,0.167
1,Albania,4307350.000,0.365
2,Armenia,22950475.000,1.943
3,Azerbaijan,14784625.000,1.251
4,Belize,150175.000,0.013


time: 8.42 ms


### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [16]:
# Alcune osservazioni sono mancanti per la colonna disburse_time.
# Si vedono le proporzioni di quesi valori mancanti rispetto a tutto il dataset
# sia per il numero di righe che per l'ammontare del prestito a cui non è associato nessuna data.

df_missing_disburse_time = df_loans[df_loans['disburse_time'].isna()]

row_missing = df_missing_disburse_time.shape[0]
loan_amount_missing = df_missing_disburse_time['loan_amount'].sum()

# Osservazioni mancanti rispetto alle righe del dataset
print(f"Righe totali del dataset: {format(df_loans.shape[0], ',d')}")
print(f"Righe mancanti: {format(row_missing, ',d')}")
print(f"Percentuale di righe mancanti: {round(((row_missing/df_loans.shape[0])*100), 3)}%\n")

# Osservazioni mancanti rispetto ai prestiti
print(f"Totale prestiti: {format(df_loans['loan_amount'].sum(), ',.3f')}")
print(f"Prestiti senza anno: {format(loan_amount_missing, ',.3f')}")
print(f"Percentuale prestiti senza anno: {format(round(((loan_amount_missing/df_loans['loan_amount'].sum())*100), 3), '.3f')}%")

Righe totali del dataset: 1,419,607
Righe mancanti: 2,813
Percentuale di righe mancanti: 0.198%

Totale prestiti: 1,181,437,300.000
Prestiti senza anno: 10,940,750.000
Percentuale prestiti senza anno: 0.926%
time: 28.4 ms


In [17]:
# Dato l'esiguo numero di valori mancanti si è scelto di non operare nessuna sostituzione di tali valori
# ma di considerarli invece allo stesso livello dei gruppi degli anni.

# Si separano le osservazioni con disburse_time presente da quelle con valore mancante
disburse_time_notna = df_loans[df_loans['disburse_time'].notna()]
disburse_time_na = df_loans[df_loans['disburse_time'].isna()]

# Per le osservazioni con disburse_time presente si estrae l'anno
disburse_time_notna['disburse_year'] = df_loans['disburse_time'].dt.year.astype('Int64')

# Per le osservazioni con disburse_time mancante si rimpiazza na missing_year
disburse_time_na['disburse_year'] = disburse_time_na['disburse_time'].fillna('missing_year')

# Si effettua il merge dei dataframe
df_loans = pd.concat([disburse_time_notna, disburse_time_na])

time: 560 ms


In [18]:
# Si effettua il group by sulla data da cui è stato estratto l'anno. Si usa dt per gestire le date nelle 
# pandas Series. Per ogni gruppo si calcolano il numero di righe e la somma dei loan
#df_loans['diburse_year'] = df_loans['disburse_time'].dt.year
df_group_country_year = df_loans.groupby(['country_name', 'disburse_year'])['loan_amount'].agg([
    ('count', 'count'), ('loan_amount', 'sum')])

# Cast degli anni ad intero
#df_group.index = df_group.index.astype(int)

# Al dataset finale si aggiunge una riga con le statistiche per le osservazioni mancanti.
#df_group = df_group.append(pd.Series({'count':row_missing, 
   #                                   'loan_amount':loan_amount_missing}, name='missing_year'))

# Si calcola la percentuale di loan amount per ogni anno riferita alla somma totale dei loans
df_group_country_year['percentage_amount'] = (df_group_country_year['loan_amount']/
                                              df_group_country_year['loan_amount'].sum())*100


df_group_country_year = df_group_country_year.reset_index()

time: 427 ms


In [59]:
# A titolo dimostrativo si motrano i dati degli Stati Uniti sia per gli anni presenti sia per quelli mancanti
df_group_country_year[df_group_country_year['country_name'] == 'United States']

,country_name,disburse_year,count,loan_amount,percentage_amount
710,United States,2009,118,645400.000,0.055
711,United States,2010,139,773725.000,0.065
712,United States,2011,222,1217650.000,0.103
713,United States,2012,398,2225850.000,0.188
714,United States,2013,1037,5326650.000,0.451
715,United States,2014,1091,5092325.000,0.431
716,United States,2015,1207,6599125.000,0.559
717,United States,2016,1102,6695750.000,0.567
718,United States,2017,1466,7037175.000,0.596
719,United States,missing_year,2400,10738350.000,0.909


time: 19 ms


### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [20]:
# Calcolo, per ogni loan, del numero dei suoi partecipanti
df_n_lenders = df_loan_lender_norm.groupby('loan_id').size().reset_index()
df_n_lenders = df_n_lenders.rename(columns= {0: 'n_lenders'})
df_n_lenders.head()

,loan_id,n_lenders
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


time: 3.76 s


In [21]:
# merge dei dataframe
df_merge = pd.merge(df_loans[['loan_id', 'loan_amount']], df_loan_lender_norm, on='loan_id')
df_merge = pd.merge(df_merge, df_n_lenders, on='loan_id')

# Calcolo dell'ammontare di ogni prestatore per ogni prestito in modo equo
df_merge['amount_per_lender'] = df_merge['loan_amount'] / df_merge['n_lenders']

# Raggruppo per lender e calcolo la somma dei prestiti dati
df_amount_per_lender = df_merge.groupby('lenders')['amount_per_lender'].sum().reset_index()

time: 1min 3s


In [22]:
df_amount_per_lender.head()

,lenders,amount_per_lender
0,000,1764.285
1,00000,1380.694
2,0002,2472.564
3,00mike00,52.632
4,0101craign0101,2623.565


time: 35.1 ms


### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [23]:
# Nel dataset c'è un numero elevato di dati mancanti per la colonna country_code
df_lenders.info(null_counts=True)

print('\n====================================================\n')

# Inoltre si vede che nel dataset dei loans la colonna country_code ha 9 valori mancanti, 
# mentre country_name non ne ha 
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349174 entries, 0 to 2349173
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   permanent_name  2349174 non-null  object
 1   city            729868 non-null   object
 2   state           635693 non-null   object
 3   country_code    890539 non-null   object
dtypes: object(4)
memory usage: 71.7+ MB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1419607 entries, 0 to 1419564
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype              
---  ------                   --------------    -----              
 0   loan_id                  1419607 non-null  int64              
 1   loan_name                1372049 non-null  object             
 2   loan_amount              1419607 non-null  float64            
 3   country_code             1419598 non-null  object             
 4   country_name             1419607 non-null  o

In [24]:
# I valori mancanti in df_loans sono associati tutti allo stato della Namibia
df_loans[df_loans['country_code'].isna()]

,loan_id,loan_name,loan_amount,country_code,country_name,planned_expiration_time,disburse_time,diff_days,disburse_year
82889,991853,Elizabeth,3325.000,NaN,Namibia,2016-01-21 02:20:06+00:00,2016-01-05 08:00:00+00:00,15.000,2016
156970,513472,Gottlieb,7150.000,NaN,Namibia,2013-02-02 00:10:01+00:00,2013-03-08 08:00:00+00:00,-35.000,2013
598087,851360,Elizabeth,4150.000,NaN,Namibia,2015-04-24 14:00:04+00:00,2015-06-03 07:00:00+00:00,-40.000,2015
684876,1068159,Theo,4000.000,NaN,Namibia,2016-06-14 14:10:06+00:00,2016-06-10 07:00:00+00:00,4.000,2016
971827,998555,Theo,3325.000,NaN,Namibia,2016-01-21 02:20:17+00:00,2016-01-05 08:00:00+00:00,15.000,2016
1134818,1147866,Ndeyapo,5000.000,NaN,Namibia,2016-10-19 01:50:05+00:00,2016-10-01 07:00:00+00:00,17.000,2016
1214923,851368,Theo,4150.000,NaN,Namibia,2015-04-24 14:00:04+00:00,2015-06-03 07:00:00+00:00,-40.000,2015
1281022,1147852,Dominic,5100.000,NaN,Namibia,2016-11-02 18:00:07+00:00,2016-10-01 07:00:00+00:00,32.000,2016
1415763,1068167,Elizabeth,3325.000,NaN,Namibia,2016-06-14 16:30:07+00:00,2016-06-10 07:00:00+00:00,4.000,2016


time: 102 ms


In [25]:
# Analogo discorso per i record dei lenders, dove tutti i valori della Namibia sono mancanti
df_lenders[df_lenders['state']=='Namibia']

,permanent_name,city,state,country_code
2063842,jakeskaova9324,Windhoek,Namibia,NaN
2158590,doreen3422,Windhoek,Namibia,NaN
2273721,ckqdm1115,Windhoek,Namibia,NaN


time: 116 ms


In [26]:
# Per evitare di perdere tutte le informazioni legate ad un intero paese,
# solo in questo caso si procede con un rimpiazzo manuale dei missing value
df_lenders.loc[df_lenders['state']=='Namibia', 'country_code'] = 'Namibia'
df_loans.loc[df_loans['country_code'].isna(), 'country_code'] = 'Namibia'

time: 192 ms


In [27]:
# Si separano i lenders con valore di country_code presente dai lenders con valore country_code mancante
df_lenders_no_na = df_lenders[df_lenders['country_code'].notna()]

# Si ottiene una serie di booleani per poter eseguire la sostistuzione dei valori mancanti successivamente
sr_lenders_na = df_lenders['country_code'].isna()

time: 379 ms


In [28]:
# Si trova la distribuzione dei lender con valore di country_code presente 
# utilizzando l'opzione normalize di value_count che calcola la frequenza relativa dei valori
sr_state_distr = df_lenders_no_na['country_code'].value_counts(normalize=True)

# Si usa la distribuzione trovata per assegnare lo stato per i valori mancanti di country_code.
# Si usa la funzione choice utilizzando come valori i codici dei paesi, come dimensione del campione la somma dei
# booleani della serie che distingue tra codice na e non na e per probabilità la distribuzione trovata.
df_lenders.loc[sr_lenders_na, 'country_code'] = np.random.choice(sr_state_distr.index, size=sum(sr_lenders_na), 
                                                                 p=sr_state_distr.values)

time: 498 ms


In [29]:
# si verifico che non ci siano valori ancanti
sum(df_lenders.country_code.isna())

0

time: 337 ms


In [30]:
# Si effettua il merge dei lenders con quanto hanno prestato in totale
df_lent = pd.merge(df_lenders, df_amount_per_lender, left_on='permanent_name', right_on='lenders')

time: 2.48 s


In [31]:
# Si sommano i prestiti raggruppati per paese
df_lent_country = df_lent.groupby('country_code')['amount_per_lender'].sum().reset_index(name='amount_lent')

time: 464 ms


In [32]:
df_lent_country.head()

,country_code,amount_lent
0,AD,8029.694
1,AE,1969121.158
2,AF,139997.469
3,AG,1706.163
4,AI,501.716


time: 6.43 ms


In [33]:
# Si calcola il totale dei prestiti ricevuti per ogni paese
df_borrowed_country = df_loans.groupby('country_code')['loan_amount'].sum().reset_index(name='borrowed')

time: 85.8 ms


In [34]:
df_borrowed_country.head()

,country_code,borrowed
0,AF,1967950.000
1,AL,4307350.000
2,AM,22950475.000
3,AZ,14784625.000
4,BA,477250.000


time: 11.4 ms


In [35]:
# Si effettua il merge dei due dataset
df_diff_lent_borrowed = pd.merge(df_lent_country, df_borrowed_country, on='country_code', how='outer')

# Si sostituiscono i valori mancanti con 0
df_diff_lent_borrowed.fillna(0, inplace=True)

# Si calcola la differenza tra il denaro prestato e quello preso in prestito
df_diff_lent_borrowed['diff_lent-borrowed'] = df_diff_lent_borrowed['amount_lent']-df_diff_lent_borrowed['borrowed']

time: 44.9 ms


In [36]:
df_diff_lent_borrowed.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed
0,AD,8029.694,0.000,8029.694
1,AE,1969121.158,0.000,1969121.158
2,AF,139997.469,1967950.000,-1827952.531
3,AG,1706.163,0.000,1706.163
4,AI,501.716,0.000,501.716


time: 14.5 ms


### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [37]:
# Come nel punto precedente, il codice dello stato della Namibia è assente
df_country_stats[df_country_stats['country_name'] == 'Namibia']

,country_name,country_code,population,population_below_poverty_line
115,Namibia,NaN,2533794,28.700


time: 19.2 ms


In [38]:
# Per non perdere le sue informazioni lo si inserisce manualmente
df_country_stats.loc[df_country_stats['country_name'] == 'Namibia', 'country_code'] = 'Namibia'

time: 7.02 ms


In [39]:
# Si effettua il merge del dataset delle informazioni degli stati con quello della differenza di prestiti 
# calcolato al punto precedente
df_money_state = pd.merge(df_diff_lent_borrowed, df_country_stats, on='country_code')

# Calcolo del rapporto richiesto
df_money_state['ratio_diff_L-B_pop'] = df_money_state['diff_lent-borrowed']/df_money_state['population']

time: 26.8 ms


In [40]:
df_money_state.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop
0,AE,1969121.158,0.000,1969121.158,United Arab Emirates,9400145,19.500,0.209
1,AF,139997.469,1967950.000,-1827952.531,Afghanistan,35530081,35.800,-0.051
2,AL,27800.918,4307350.000,-4279549.082,Albania,2930187,14.300,-1.461
3,AM,162134.274,22950475.000,-22788340.726,Armenia,2930450,32.000,-7.776
4,AO,39000.678,0.000,39000.678,Angola,29784193,40.500,0.001


time: 37.1 ms


In [41]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop'].idxmax()]]

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop
113,NO,20671010.306,0.000,20671010.306,Norway,5305383,nan,3.896


time: 23.8 ms


### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [42]:
# La colonna population_below_poverty_line è espressa in percentuale,
# Si calcola la percentuale di persone sopra la soglia di poverà
df_money_state['population_above_poverty_line'] = 100 - df_money_state['population_below_poverty_line']

# Si calcola la quantità di popolazione al di sotto della fascia di povertà utilizzando la popolazione totale 
# e la percentuale calcolata sopra
pop_above_poverty_line = (df_money_state['population']*df_money_state['population_above_poverty_line'])/100

# Si calcola il rapport richiesto
df_money_state['ratio_diff_L-B_pop_above_poverty'] = df_money_state['diff_lent-borrowed']/pop_above_poverty_line

time: 6.36 ms


In [43]:
df_money_state.head()

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop,population_above_poverty_line,ratio_diff_L-B_pop_above_poverty
0,AE,1969121.158,0.000,1969121.158,United Arab Emirates,9400145,19.500,0.209,80.500,0.260
1,AF,139997.469,1967950.000,-1827952.531,Afghanistan,35530081,35.800,-0.051,64.200,-0.080
2,AL,27800.918,4307350.000,-4279549.082,Albania,2930187,14.300,-1.461,85.700,-1.704
3,AM,162134.274,22950475.000,-22788340.726,Armenia,2930450,32.000,-7.776,68.000,-11.436
4,AO,39000.678,0.000,39000.678,Angola,29784193,40.500,0.001,59.500,0.002


time: 18.7 ms


In [44]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop_above_poverty'].idxmax()]]

,country_code,amount_lent,borrowed,diff_lent-borrowed,country_name,population,population_below_poverty_line,ratio_diff_L-B_pop,population_above_poverty_line,ratio_diff_L-B_pop_above_poverty
24,CA,102261495.315,50000.000,102211495.315,Canada,36624199,9.400,2.791,90.600,3.080


time: 21.8 ms


### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Ci sono tre possibili casi da considerare:
1. i loans per cui le date di disburse_time e planned_expiration_time sono presenti e sono coerenti, 
   cioè quando planned_expiration_time > disburse_time
2. i loans in cui le due date sono presenti ma si ha planned_expiration_time > disburse_time
3. i loans per cui almeno una delle due date non è presente

Il primo passaggio consiste ne separare le righe del caso 3 dalle altre, per questi loans non è possibile 
calcolare i dati richiesti, si sostiuiranno i dati mancanti con 0.

Il passo successivo sarà distinguere i loans appartenenti al caso 1 da quelli appartenenti al caso 2,
nel primo caso si procederà per il calcolo dell'ammontere per anno.

Per i lenders che appartengono al caso 2, dato che non è possibile ricevere un prestito dopo la data della 
sua scadenza, si considerano i due valori come identici, in particolare uguali a disburse_time.

In [45]:
# Si separano i lender con valori mancanti dagli altri
df_loans_na = df_loans[df_loans['disburse_time'].isna() | df_loans['planned_expiration_time'].isna()]
df_loans_no_na = df_loans.dropna(subset=['disburse_time', 'planned_expiration_time'])

time: 871 ms


In [46]:
# info dei due dataframe
df_loans_na.info()
print('\n=================================================================\n')
df_loans_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374645 entries, 64 to 1419564
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   loan_id                  374645 non-null  int64              
 1   loan_name                358121 non-null  object             
 2   loan_amount              374645 non-null  float64            
 3   country_code             374645 non-null  object             
 4   country_name             374645 non-null  object             
 5   planned_expiration_time  2811 non-null    datetime64[ns, UTC]
 6   disburse_time            371832 non-null  datetime64[ns, UTC]
 7   diff_days                0 non-null       float64            
 8   disburse_year            374645 non-null  object             
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(4)
memory usage: 28.6+ MB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044962 entr

In [47]:
# Si gestiscono i casi in cui le due date non sono corenti
loans_inc = df_loans_no_na['disburse_time'] > df_loans_no_na['planned_expiration_time']

df_loans_no_na.loc[loans_inc, 'planned_expiration_time'] = df_loans_no_na.loc[loans_inc, 'disburse_time']

time: 80.8 ms


In [48]:
def get_amount_x_year(row):
    '''
    Riceve in input una riga del dataset e calcola, per ogni anno tra disburse_time e planned_expiration_time,
    la quantità di giorni di quell'anno. Nel caso di disburse_time considera i giorni dalla data fino alla fine 
    dell'anno, nel caso di planned_expiration_time considera i giorni passati dall'inizio dell'anno alla data in 
    questione. Ritorna un dizionario le cui chiavi sono gli anni all'interno del periodo considerato e i valori
    sono l'ammontare del loan per quell'anno dell'anno calcolati come descritto.
    ...

    Parameters
    ----------
    row : pandas.core.series.Series
        La riga del dataset
    
    Returns
    -------
    dict
        dizionario di con le informazioni del loan(id, name, amount), i giorni del prestito e per ogni anno
        l'importo in base ai giorni trascorsi
    '''
    
    # Si estraggono gli anni in mezzo
    planned = row['planned_expiration_time']
    disburse = row['disburse_time']
    
    # Calcolo degli anni tra disburse e planned esclusi loro stessi
    range_year = list(range(disburse.year + 1, planned.year))
    
    tot_amount = row['loan_amount']
    tot_days = (planned - disburse).days
    
    amount_x_year = dict()
    
    # Si inseriscono i dati iniziali del prestito
    amount_x_year.update({
        'loan_id': row['loan_id'], 
        'loan_name': row['loan_name'], 
        'loan_amount': tot_amount,
        'days_loan': tot_days
    })
    
    # Se planned e disburse hanno lo stesso anno, considero solo la loro differenza in giorni
    if planned.year == disburse.year:
        amount_x_year[planned.year] = tot_amount
        
    # Altrimenti se gli anni sono differenti si aggiungono entrambi
    else:
        # Per disburse devo calcolare quelli che rimangono fino alla fine dell'anno,
        # Si calcolano i giorni totali dell'anno in questione che vengono tolti ai giorni dell'anno passati 
        # fino a disburse
        days_x_year = pd.Period('{}-12-31'.format(disburse.year)).dayofyear - disburse.dayofyear
        
        # Calcolo dell'ammontare per l'anno disburse
        amount_x_year[disburse.year] = tot_amount*(days_x_year/tot_days)
        
        # Per planned si calcolano i giorni passati fino alla data in questione
        # e l'ammontare per l'anno planned
        amount_x_year[planned.year] = tot_amount*((planned.dayofyear-1)/tot_days)
        
        # Per gli anni all'interno del range aggiungo coppia (anno, ammontere per quell'anno) al dict
        if range_year:
            amount_x_year.update({
                k: tot_amount*(pd.Period('{}-12-31'.format(k)).dayofyear/tot_days) for k in range_year
            })
            
    return amount_x_year


time: 14.4 ms


In [49]:
# Calcolo del prestito per ogni anno
res = df_loans_no_na.apply(lambda x: get_amount_x_year(x), axis=1)

time: 6min 30s


In [50]:
# Creazione del dataframe e riordino delle colonne
df_res = pd.DataFrame(list(res))
df_res.columns = list(map(str, df_res.columns))
cols_reorder = ['loan_id', 'loan_name', 'loan_amount', 'days_loan', '2011', 
                '2012', '2013', '2014', '2015', '2016', '2017', '2018']
df_res = df_res.reindex(cols_reorder, axis=1)


time: 3.12 s


In [51]:
df_res.head()

,loan_id,loan_name,loan_amount,days_loan,2011,2012,2013,2014,2015,2016,2017,2018
0,657307,Aivy,125.000,53,nan,nan,21.226,103.774,nan,nan,nan,nan
1,657259,Idalia Marizza,400.000,96,nan,nan,45.833,350.000,nan,nan,nan,nan
2,658010,Aasia,400.000,37,nan,nan,nan,400.000,nan,nan,nan,nan
3,659347,Gulmira,625.000,34,nan,nan,nan,625.000,nan,nan,nan,nan
4,656933,Ricky\t,425.000,57,nan,nan,104.386,320.614,nan,nan,nan,nan


time: 19.5 ms


In [52]:
# Gestione dei record con almeno una delle due date mancanti, si creano le colonne degli anni e dei giorni passati
# che contengono solo na
df_na_sel = df_loans_na[['loan_id', 'loan_name','loan_amount']]
df_na_sel[['days_loan', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']] = pd.DataFrame(
    [np.full([9], np.nan)], index=df_na_sel.index)

time: 83.3 ms


In [53]:
# Concatenazione dei due dataset
df_res = pd.concat([df_res, df_na_sel])

time: 964 ms


In [54]:
df_res.head()

,loan_id,loan_name,loan_amount,days_loan,2011,2012,2013,2014,2015,2016,2017,2018
0,657307,Aivy,125.000,53.000,nan,nan,21.226,103.774,nan,nan,nan,nan
1,657259,Idalia Marizza,400.000,96.000,nan,nan,45.833,350.000,nan,nan,nan,nan
2,658010,Aasia,400.000,37.000,nan,nan,nan,400.000,nan,nan,nan,nan
3,659347,Gulmira,625.000,34.000,nan,nan,nan,625.000,nan,nan,nan,nan
4,656933,Ricky\t,425.000,57.000,nan,nan,104.386,320.614,nan,nan,nan,nan


time: 18.7 ms


In [55]:
df_res[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']].sum().reset_index(name ='Total Amount')

,index,Total Amount
0,2011,631678.344
1,2012,113276514.286
2,2013,124798037.059
3,2014,152880159.407
4,2015,159519008.035
5,2016,158988480.604
6,2017,172242358.528
7,2018,5700798.386


time: 1.98 s
